In [1]:
# Program to connect to the tweeter api and collect tweets
# related to AI. For the sake of this experiment 
import tweepy as tw

# Important tweeter tokens for auth
consumer_key= 'rtOHXdgu2S8SIFctVfF0yhXcY'
consumer_secret= 'l8E5AIArXxZvr2idFTCzoLDkjqssLVIeo3TaUSyObqH0tQ7KsE'
access_token= '185329008-rIYt3Y8HBkgBVSdYcy6iTMkXiUXFF3cSJkjuCZU6'
access_token_secret= 'qKIz3VOjOus4mvNNg0JYGbaMHncPgLqajhUxJfiUBTHbL'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


In [30]:
# Collect tweets
search_words = ['#ai', '#artificial intelligence', '#artificialintelligence']
date_since = '2020-11-16'

# Since we are purely using this to evaluate the nltk/textblob 
# for sentiment analysis let's use a sample set of 100
sample_tweets = 200
tweets = []

for search_word in search_words:
    _tweet_obj = tw.Cursor(api.search,
                           q=search_word,
                           lang="en",
                           since=date_since, tweet_mode='extended').items(sample_tweets)
    for tweet in _tweet_obj:
        # When using the extended tweet_mode, the object attr changes to
        # full_text
        tweets.append(tweet.full_text)
    break
        
print(tweets)

['The latest The Hacking Daily! https://t.co/FPUOLfy5le Thanks to @Lemmiwinks_III @KatiWeis #ai #machinelearning', '5 Ways Augmented Reality Apps Are Transforming Our Everyday Lives https://t.co/kqrjmhxpWB #AI #MachineLearning #DataScience #ArtificialIntelligence\n\nTrending AI/ML Article Identified &amp; Digested via Granola; a Machine-Driven RSS Bot by Ramsey Elbasheer https://t.co/75g99USLUl', 'RT @EasternStyle1: A robot out of control.#Programming #IoT #AI #100DaysOfCode #MachineLearning #ML #Airdrop #Python #CloudComputing #Analy…', 'RT @Everyday_MBA: [podcast] Leon Kuperman discusses AI-driven cloud optimization and how organizations can reduce their cloud spending w #A…', 'RT @spxbot: The current wave viewed by r.Virgeel  #artificialintelligence #ai #spx #sp500 a.i. for investors and traders https://t.co/3DobV…', 'RT @mvollmer1: A simulated video 🎥 of eye 👁 movements via @EasternStyle1 \n\n#Programming #AI #100DaysOfCode #MachineLearning #ML #Python #An…', 'RT @DarrabEducation: 

In [31]:
print(len(tweets))

200


In [32]:
# First, let's use NLTK's sentiment-analysis across
# collected tweets
import pandas as pd
import nltk
import re
# nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

df = pd.DataFrame(columns=['Tweet', 'Sentiment'])
                           
for tweet in tweets:
    # Remove tweets with urls
    if 'https' in tweet:
        continue
    # Substitute retweet data
    tweet = re.sub('RT \@.*: ', '', tweet, 1)
    if (df['Tweet'] == tweet).any():
        continue
    print(tweet)
    score = sia.polarity_scores(tweet)['compound']
    if score < 0:
        sentiment = 'Negative'
    elif score == 0:
        sentiment = 'Neutral'
    elif score > 0:
        sentiment = 'Positive'
    # Finally, add the row to the data-frame
    df = df.append({'Tweet': tweet, 'Sentiment': sentiment}, ignore_index=True)

df.head()

A robot out of control.#Programming #IoT #AI #100DaysOfCode #MachineLearning #ML #Airdrop #Python #CloudComputing #Analy…
[podcast] Leon Kuperman discusses AI-driven cloud optimization and how organizations can reduce their cloud spending w #A…
A simulated video 🎥 of eye 👁 movements via @EasternStyle1 

#Programming #AI #100DaysOfCode #MachineLearning #ML #Python #An…
TechGirls Leadership Program in the #USA 
Coverage: Fully Funded
Levels: Bachelors 
Nationality: All
Deadline 15.01.20…
Deep Vision #01053

#DeepVision #DeepVisionAI #CNFT #CNFTs #Cardano #CardanoCommunity #NFT #NFTs #NFTArt #AI #ArtificialInte…
Researchers describe how specific facial features distinguish a child's face from an adult's face. The findings may he…
Through digital transformation enabled by artificial intelligence (AI), Healthcare companies can tap into underutilize…
Greetings from @SpaceTechExpoEU! Thank you all space guys for your interest in our solutions and your attendance at the panel…
What is natural 

,Tweet,Sentiment
0,A robot out of control.#Programming #IoT #AI #...,Neutral
1,[podcast] Leon Kuperman discusses AI-driven cl...,Positive
2,A simulated video 🎥 of eye 👁 movements via @Ea...,Neutral
3,TechGirls Leadership Program in the #USA \nCov...,Neutral
4,Deep Vision #01053\n\n#DeepVision #DeepVisionA...,Positive


In [27]:
df.to_csv('ai_tweets.csv', index=False)

In [5]:
import pandas as pd
# Let's calculate the kappa co-efficient based on the obtained results
df_mturks = pd.read_csv('ai_tweets_final.csv')
df_mturks.head()

,Tweet,NLTK-Sentiment-Analyser,Mturk-1,Mturk-2
0,#Walmart teaming up with @zipline to launch an...,Neutral,Neutral,Neutral
1,Mathematics is Magic!\n#AI #MachineLearning #d...,Neutral,Positive,Positive
2,I Made a Git Cheat-Sheet by @elmiracodes. #Big...,Neutral,Neutral,Positive
3,Will combination of #AI &amp; ITSM continue to...,Positive,Neutral,Neutral
4,[#Innovation] This Scottish island has the wor...,Neutral,Neutral,Neutral


In [8]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(df_mturks['Mturk-1'], df_mturks['Mturk-2'], 
                  labels=['Positive', 'Negative', 'Neutral'])

# The manual calculation coroborates with the result of cohen_kappa_score()
# A negative score shows a disagrement between the two Turkers.

-0.054117647058823604

In [9]:
# Repeat this for other combinations
cohen_kappa_score(df_mturks['NLTK-Sentiment-Analyser'], df_mturks['Mturk-1'], 
                  labels=['Positive', 'Negative', 'Neutral'])

0.04622871046228705

In [10]:
cohen_kappa_score(df_mturks['NLTK-Sentiment-Analyser'], df_mturks['Mturk-2'], 
                  labels=['Positive', 'Negative', 'Neutral'])

-0.1454545454545455